In [8]:
import base64
import os
import pandas as pd
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from email.utils import parsedate_to_datetime


# Authentication function
def authenticate_gmail_api():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json')
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', ['https://www.googleapis.com/auth/gmail.readonly'])
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds


def download_emails(service, max_emails=10000):
    result = service.users().messages().list(userId='me', maxResults=max_emails).execute()
    messages = result.get('messages', [])
    return messages

In [9]:
def process_emails(emails, service):
    df = pd.DataFrame(columns=['Date', 'From', 'Subject', 'Body'])

    for email in emails:
        try:
            msg_id = email['id']
            # Retrieve the email using 'full' format for easier parsing
            message = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
            payload = message['payload']

            headers = {header['name']: header['value'] for header in payload.get('headers', [])}
            date = headers.get('Date', '')
            from_email = headers.get('From', '')
            subject = headers.get('Subject', '')

            # Parse the email date
            if date:
                date_iso = parsedate_to_datetime(date).strftime('%Y-%m-%dT%H:%M:%S')
            else:
                date_iso = ''

            body = extract_body(payload)

            if body:
                df = pd.concat([df, pd.DataFrame({'Date': date_iso, 'From': from_email, 'Subject': subject, 'Body': body}, index=[len(df)])], ignore_index=True)
            else:
                print("Skipping email due to empty body")
        except Exception as e:
            print(f"Skipping email due to error: {e}")

    return df


def extract_body(payload):
    # Function to extract the body content from the email payload
    body = ''
    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                body = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8', errors='ignore')
                break
            elif 'parts' in part:
                # Recursively search through nested parts
                body = extract_body(part)
                if body:
                    break
    elif payload['mimeType'] == 'text/plain':
        body = base64.urlsafe_b64decode(payload['body']['data']).decode('utf-8', errors='ignore')
    return body


# Main script to authenticate and process emails
service = build('gmail', 'v1', credentials=authenticate_gmail_api())
emails = download_emails(service)
email_df = process_emails(emails, service)
email_df.to_csv('emails.csv', index=False)
print("Emails saved to CSV file.")

Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping email due to empty body
Skipping e

In [6]:

import pandas as pd
import subprocess
import json

# Load the email data
df = pd.read_csv('emails.csv')

# Optionally strip whitespace from column names
df.columns = df.columns.str.strip()

# Print columns to verify
print("Columns in DataFrame:", df.columns.tolist())

# Lists to store the results
summaries = []
products_list = []
categories = []

# Function to interact with Ollama
def query_ollama(prompt, model='llama3.2'):
    try:
        command = ['ollama', 'run', model]
        process = subprocess.Popen(
            command,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        stdout, stderr = process.communicate(input=prompt)
        if process.returncode != 0:
            print(f"Error querying Ollama: {stderr}")
            return None
        return stdout.strip()
    except Exception as e:
        print(f"Exception during querying Ollama: {e}")
        return None

# Prompt template
prompt_template = """
You are an AI assistant that processes emails.

Given the following email content:

---
{email_content}
---

Please perform the following tasks:

1. **Summary**: Provide a brief summary of the email in 2-3 sentences.

2. **Products Mentioned**: List any products mentioned in the email. If none, state "None".

3. **Category**: Classify the email into one of the following categories:
   - Purchase
   - Information/News/Blog
   - Personal Communication

Format your response as a JSON object with the following keys: "summary", "products", and "category".

Example:

{{
  "summary": "...",
  "products": ["Product A", "Product B"],
  "category": "Purchase"
}}
"""

for index, row in df.iterrows():
    email_content = row['Body']

    if not isinstance(email_content, str) or not email_content.strip():
        print(f"Skipping email at index {index} due to empty body")
        summaries.append('')
        products_list.append([])
        categories.append('')
        continue

    prompt = prompt_template.format(email_content=email_content)
    response = query_ollama(prompt)
    if response:
        try:
            # Attempt to parse the response as JSON
            result = json.loads(response)
            summaries.append(result.get('summary', ''))
            products_list.append(result.get('products', []))
            categories.append(result.get('category', ''))
        except json.JSONDecodeError:
            # If JSON parsing fails, try to extract information manually
            summary = ''
            products = []
            category = ''
            
            # Simple parsing logic (you may need to adjust this based on the actual output format)
            lines = response.split('\n')
            for line in lines:
                if line.startswith('Summary:'):
                    summary = line.split(':', 1)[1].strip()
                elif line.startswith('Products:'):
                    products = [p.strip() for p in line.split(':', 1)[1].strip().split(',')]
                elif line.startswith('Category:'):
                    category = line.split(':', 1)[1].strip()
            
            summaries.append(summary)
            products_list.append(products)
            categories.append(category)
    else:
        summaries.append('')
        products_list.append([])
        categories.append('')

# Update the DataFrame
df['summary'] = summaries
df['products'] = products_list
df['category'] = categories

# Save the updated CSV
df.to_csv('email_updated.csv', index=False)


Columns in DataFrame: ['Date', 'From', 'Subject', 'Body']
